In [0]:
import requests
import numpy as np
import pandas as pd
import os
import re
import json
from pandas import DataFrame


def data_crawlling(code):

    url = "https://wisefn.finance.daum.net/v1/company/cF1001.aspx?cmp_cd=" +code +  "&frq=0&rpt=1&finGubun=MAIN"
    html =requests.get(url).text
    
    # 컬럼명 : 기준년월 추출
    match1 = re.search(r'var changeFin = (.+?);',html, re.S )
    json_string1 = match1.group(1)
    col_name = json.loads(json_string1)

    col_name = col_name[0]+ col_name[1]
    date_name = []
    y_name = ['type']
    q_name = ['type']
    for i in range(0,len(col_name)) :
        if i < 4 :
            y_name.append( col_name[i][:7])
        else :
            q_name.append( col_name[i][:7] )


    # 재무데이터 추출
    match2 = re.search(r'var changeFinData = (.+?);',html, re.S )
    json_string2 = match2.group(1)
    fin_data = json.loads(json_string2)


    y_data = fin_data[0][0] + fin_data[1][0] + fin_data[2][0] + fin_data[3][0]
    q_data = fin_data[0][1] + fin_data[1][1] + fin_data[2][1] + fin_data[3][1]

    y_df = pd.DataFrame(y_data)    
    q_df = pd.concat( [y_df.iloc[:,0], pd.DataFrame(q_data) ], axis= 1)
    
    y_df.columns = y_name
    q_df.columns = q_name
    
    return y_df, q_df




In [0]:
def make_table(df, code):
    

    #컬럼필터링 & Transpos
    temp_data_T = df.T

    # 종목컬럼추가
    temp_data_T.columns = temp_data_T.iloc[0]
    temp_data_T = temp_data_T.drop("type", axis=0)
    temp_data_T['code'] =  [code]*len(temp_data_T)
    col_1 = temp_data_T.columns.tolist()


    col_2 = col_1[-1:] + col_1[:-1]
    temp_data_T = temp_data_T[col_2]
    temp_data_T
    temp_data_T = temp_data_T.reset_index()
#     temp_data_T = temp_data_T.reset_index(drop = True)
    temp_data_T = temp_data_T.rename(columns={'index' : 'DATE'})

    return temp_data_T

In [0]:
# 종목코드 리스트 가져오기
# Cleansing_fin_data 에서 만든 stock_df.csv 가져오기 
stock_list = pd.read_csv("stock_df_compl.csv",  converters={'종목코드': str})
code_list = stock_list['종목코드'].tolist()

In [0]:
# 크롤링 함수 수행
i = 0
for code_i in code_list:
        
        #데이터 크롤링
        y_df, q_df = data_crawlling(code_i)
        
        #연간데이터 / 분기데이터
        if i == 0 :
            year_data = make_table(y_df, code_i  )
            qauter_data = make_table(q_df, code_i  )
            i = 1
        else:
            year_data = pd.concat([year_data, make_table(y_df, code_i  )])
            qauter_data = pd.concat([qauter_data, make_table(q_df, code_i  )])
            

In [0]:
year_data.to_csv( "year_data.csv", index = False, encodeing = 'ms949')
qauter_data.to_csv("quater_data.csv", index = False, encodeing = 'ms949')